In [20]:
# preferível esta técnica proque embedding é mais caro
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import PromptTemplate
from langchain_openai import ChatOpenAI
from dotenv import load_dotenv
import os

load_dotenv(dotenv_path='.env')
openai_api_key = os.getenv("OPENAI_API_KEY")

classification_template = PromptTemplate.from_template(
    """Você deve classificar uma pergunt
    
    Dada a pergunta do usuário abaixo, classifique-a como sendo sobre `Carro`, `Restaurante` ou `Tecnologia`.

    <Se a pergunta for sobre mecânica de carros, modelos ou tecnologia automotiva, classifique-a como 'Carro'>
    <Se a pergunta for sobre piadas, classifique-a como 'Piada'>
    <Se a pergunta for sobre patentes, classifique-a como 'Patente'>
    
    <question>
    {question}
    </question>
    
    Classificação:"""
)

classification_chain = classification_template | ChatOpenAI() | StrOutputParser()

In [34]:
import re

carro_template = """Você é um especialista em automóveis. Você tem amplo conhecimento sobre mecânica de carros, \
modelos e tecnologia automotiva. Você fornece respostas claras e úteis sobre carros.\
\
Pergunta:\
{query}"""

piada_template = """Você é um comediante,alegre e divertido. Dada uma pergunta crie uma piada sobre a pergunta \
\
Pergunta:\
{query}"""

patent_template = """Você é um assistente que responde perguntas sobre pedidos de patente, levando em conta o contexto {contexto} \
\
Pergunta:\
{query}"""

# teste expressão regular online https://regex101.com/

def extrair_numero_pedido(texto):
    padrao = r"(PI|MU|C1|C2|C3|C4|C5|C6|C7|C8|C9)\s*\d{7}(?:-\d)?|(?:BR\s*)?(?:\d{2}\s*)?(?:\d{4}\s*)?\d{6}(?:-\d)?"
    match = re.search(padrao, texto)
    if match:
        numero_pedido = match.group()
        numero_pedido = numero_pedido.replace(" ", "").upper().strip()
        return numero_pedido
    else:
        return None
        
def prompt_router(input):
    classification = classification_chain.invoke({"question": input["query"]})
   
    if classification == "Carro":
        print("Pergunta relativa a carros")
        return PromptTemplate.from_template(carro_template)
    elif classification == "Restaurante":
        print("Pergunta relativa a restaurantes")
        return PromptTemplate.from_template(piada_template)
    elif classification == "Patente":
        print("Pergunta relativa a patentes")
        numero = extrair_numero_pedido(input["query"])
        if numero:
            contexto = f"O pedido {numero} teve carta patente concedida em 2024" 
        else:
            contexto = "Informações adicionais sobre o pedido não foram encontradas."
        return PromptTemplate.from_template(patent_template).format(query=input["query"], contexto=contexto)
    else:
        print("Unexpected classification:", classification)
        return None

query = "Qual o andamento do pedido de patente PI8900908?"
query = "Qual o andamento do pedido de patente ?"
input_query = {"query": query, "contexto": ""}
prompt = prompt_router(input_query)

Pergunta relativa a patentes


In [35]:
from langchain_core.runnables import RunnableLambda, RunnablePassthrough
chain = (
    {"query": RunnablePassthrough(), "contexto": RunnablePassthrough()}
    | RunnableLambda(prompt_router)
    | ChatOpenAI()
    | StrOutputParser()
)

In [36]:
chain.invoke(query)

Pergunta relativa a patentes


'Para verificar o andamento de um pedido de patente, você precisará acessar o site do Instituto Nacional da Propriedade Industrial (INPI) ou do escritório de patentes do país em questão. Lá você poderá inserir o número do pedido de patente e acompanhar o status atual do processo, como se está em análise, se foi concedido ou se houve alguma objeção. Se precisar de mais ajuda, estou à disposição.'

In [37]:
chain.invoke("qual o concorrente do Compass ?")

Pergunta relativa a carros


'O Jeep Compass tem vários concorrentes diretos no mercado de SUVs compactos. Alguns dos principais concorrentes do Jeep Compass incluem o Ford Escape, Chevrolet Equinox, Honda CR-V, Toyota RAV4, Mazda CX-5, Nissan Rogue e Volkswagen Tiguan. Cada um desses modelos oferece diferentes recursos, desempenho e preços, então é importante comparar todas as opções antes de tomar uma decisão de compra.'

In [12]:
# https://www.youtube.com/watch?v=JbiTQVKvNew&t=597s
# Fazendo um Sistema de Agentes com I.A do Zero Sem CrewAI
# https://github.com/inteligenciamilgrau/videos_tutoriais/tree/main/crewai_sem_crewai

import os
import requests
import json
from dotenv import load_dotenv

# Load environment variables from .env file
app_dir = os.path.join(os.getcwd(), "app")
load_dotenv(os.path.join(app_dir, ".env"))

def send_message(prompt, sistema = "", json_format = False):

    api_key = os.getenv("OPENAI_API_KEY")  # Get the API key from the environment
    url = "https://api.openai.com/v1/chat/completions"
    headers = {
        "Authorization": f"Bearer {api_key}",
        "Content-Type": "application/json"
    }

    formato = "text"
    if json_format:
        formato = "json_object"

    mensagem = []
    if sistema != "":
        mensagem.append({"role": "system", "content": sistema})
    mensagem.append({"role": "user", "content": prompt})

    data = {
        "model": "gpt-4o-mini",  # Ensure to specify the correct model
        "messages": mensagem,
        "max_tokens": 512,  # You can adjust this as needed
        "response_format": { "type": formato },
    }

    response = requests.post(url, headers=headers, json=data)

    if response.status_code == 200:
        response_json = response.json()
        if json_format:
            return json.loads(response_json['choices'][0]['message']['content'])
        return response_json['choices'][0]['message']['content']
    else:
        print(f"Error: {response.status_code}, {response.text}")
        return None

In [13]:
# from comunicacao import send_message


formato_json = True
sistema = "Seu nome é Felipa"
mensagem = """
Bom dia, qual seu nome? Responda num JSON
"""

# Example usage:
response = send_message(mensagem, sistema, formato_json)
print(response)

{'nome': 'Felipa'}


In [15]:
#from comunicacao import send_message
import os

formato_json = True
sistema = """Seu nome é Felipa.
Caso você receba uma mensagem pedindo por algum código, responda tipo 1.
Nos outros casos responda tipo 0.

Formato da resposta deve ser um JSON conforme o modelo:
{ "tipo" : 1 }
"""
mensagem = "Bom dia, qual seu nome?"
mensagem = "Você poderia gerar um código em python para imprimir 'ola mundo'?"
mensagem = "Você poderia gerar um código em python para imprimir de 0 a 100?"
mensagem = "gere o jogo da cobrinha em python."
#mensagem = "Bom dia, qual a capital do Brasil?"

# Example usage:
response = send_message(mensagem, sistema, formato_json)
print(response)

if response['tipo'] == 0:
    formato_json = False
    sistema = """Seu nome é Assistente Mil Grau"""
    response = send_message(mensagem, sistema, formato_json)
    print(response)

elif response['tipo'] == 1:
    sistema = """gere um código em python e responda no formato JSON
    Exemplo:
    {"code": "import ..."}
    """
    response = send_message(mensagem, sistema, formato_json)

    print(response['code'])

    def save_and_execute_python_code(code_string, filename='script.py'):
        # Salvar o código no arquivo especificado
        with open(filename, 'w', encoding="utf-8") as file:
            file.write(code_string)

        print("### RODANDO ###")

        # Executar o arquivo
        os.system(f'python {filename}')

        print("### RODOU ###")

    save_and_execute_python_code(response['code'])

{'tipo': 1}


JSONDecodeError: Unterminated string starting at: line 1 column 10 (char 9)

In [16]:
#from comunicacao import send_message
import os
import re

# teste expressão regular online https://regex101.com/

def extrair_numero_pedido(texto):
    padrao = r"(PI|MU|C1|C2|C3|C4|C5|C6|C7|C8|C9)\s*\d{7}(?:-\d)?|(?:BR\s*)?(?:\d{2}\s*)?(?:\d{4}\s*)?\d{6}(?:-\d)?"
    match = re.search(padrao, texto)
    if match:
        numero_pedido = match.group()
        numero_pedido = numero_pedido.replace(" ", "").upper().strip()
        return numero_pedido
    else:
        return None
        
formato_json = True
sistema = """Seu nome é Felipa.
Caso você receba uma mensagem pedindo pelo status de um pedido de patente, responda tipo 1.
Nos outros casos responda tipo 0.

Formato da resposta deve ser um JSON conforme o modelo:
{ "tipo" : 1 }
"""
mensagem = "Bom dia, qual a capital do Brasil?"
mensagem = "Bom dia, qual a status do pedido de patente PI 9008989-9?"
mensagem = "Boa tarde, me informe a situação de BR102023024151-9?"
#mensagem = "Bom dia, qual a capital do Brasil?"

# Example usage:
response = send_message(mensagem, sistema, formato_json)
print(response)

if response['tipo'] == 0:
    formato_json = False
    sistema = """Patentes promovem o desenvolvimento do país"""
    response = send_message(mensagem, sistema, formato_json)
    print(response)

elif response['tipo'] == 1:
    sistema = """informe o status do pedido
    Exemplo:
    {"pergunta": mensagem,
     "resposta": "O pedido de patente está concedido teve um despacho 161 publicado na RPI de 13 de junho de 2024"}
    """
    print(mensagem)
    print(extrair_numero_pedido(mensagem))
    #response = send_message(mensagem, sistema, formato_json)

    #print(response['code'])


{'tipo': 1}
Boa tarde, me informe a situação de BR102023024151-9?
BR102023024151-9


In [17]:
#Patentes de Invenção: 
#10 – para pedidos depositados por nacionais e via CUP (antigo PI); 
#11 – para pedidos depositados via PCT (antigo PI PCT); 
#12 – para pedidos divididos (antigo PI); 
#13 – para certificado de adição (antigo C1, C2, etc). 
#Patentes de Modelo de Utilidade: 
#20 – para pedidos depositados por nacionais e via CUP (antigo MU); 
#21 – para pedidos depositados via PCT (antigo MU PCT); 
#22 – para pedidos divididos (antigo MU).

def extrair_digito_verificador(numero_pedido):
    # Remover espaços e prefixo "BR" se presente
    numero_pedido = numero_pedido.strip().replace(" ", "").upper()
    
    # Encontrar a posição do hífen e extrair o dígito verificador
    if '-' in numero_pedido:
        partes = numero_pedido.split('-')
        if len(partes) == 2 and len(partes[1]) == 1 and partes[1].isdigit():
            digito_verificador = partes[1]
        else:
            return -1 # ("O formato do número do pedido está incorreto.")
    else:
        return -2 # ("O número do pedido não contém um hífen com o dígito verificador.")
    
    return int(digito_verificador)
    
    
def calcular_digito_verificador(numero_pedido):
    # Remover espaços e juntar os números do pedido em uma única string
    numero_pedido = ''.join(numero_pedido.split())

    numero_pedido = numero_pedido.replace(" ", "").upper().strip()

    prefixos = ("BR", "PI", "MU", "C1", "C2", "C3", "C4", "C5", "C6", "C7", "C8", "C9")
    if numero_pedido.startswith(prefixos):
        numero_pedido = numero_pedido[2:]
        
    digito_verificador = -1

    numero_pedido = numero_pedido.split('-')[0] # ignore o que ve depois do hofen, se houver
    
    # Verificar se o número tem o formato correto (12 dígitos)
    if len(numero_pedido) == 12 and numero_pedido.isdigit():

        prefixos = ("10", "11", "12", "13", "20", "21", "22")
        if not numero_pedido.startswith(prefixos):
            return -2

        # https://www.uece.br/agin/noticias/inpi-veja-como-calcular-o-digito-verificador-na-nova-numeracao-da-dirpa-e-da-dicig/
        # Inverter o número do pedido
        numero_invertido = numero_pedido[::-1]
        
        # Inicializar variáveis
        soma = 0
        multiplicador = 2
        
        # Calcular soma dos produtos dos dígitos pelo multiplicador
        for digito in numero_invertido:
            soma += int(digito) * multiplicador
            multiplicador += 1
            if multiplicador > 9:
                multiplicador = 2
        
        # Calcular o resto da divisão da soma por 11
        resto = soma % 11
        
        # Calcular o dígito verificador
        digito_verificador = 11 - resto
        if digito_verificador == 10 or digito_verificador == 11:
            digito_verificador = 0

    if len(numero_pedido) == 7 and numero_pedido.isdigit():
        
        # Inverter o número do pedido
        numero_invertido = numero_pedido[::-1]
        
        # Inicializar variáveis
        soma = 0
        multiplicador = 2
        
        # Calcular soma dos produtos dos dígitos pelo multiplicador
        for digito in numero_invertido:
            soma += int(digito) * multiplicador
            multiplicador += 1
            if multiplicador > 9:
                multiplicador = 2
        
        # Calcular o resto da divisão da soma por 11
        resto = soma % 11
        
        # Calcular o dígito verificador
        digito_verificador = 11 - resto
        if digito_verificador == 10 or digito_verificador == 11:
            digito_verificador = 0

        #digito_verificador = -3

    return digito_verificador

# Exemplo de uso
numero_pedido = "BR102023024151-9" # digito = 4
numero_pedido = "PI9808989" # digito 7
numero_pedido = "PI 0605203-7"
digito = calcular_digito_verificador(numero_pedido)
print("Dígito verificador:", digito)

digito_user=extrair_digito_verificador(numero_pedido)
print("Dígito usuário:", digito_user)


Dígito verificador: 7
Dígito usuário: 7


In [18]:
import requests
from datetime import datetime

month_names = {
    1: "janeiro",
    2: "fevereiro",
    3: "março",
    4: "abril",
    5: "maio",
    6: "junho",
    7: "julho",
    8: "agosto",
    9: "setembro",
    10: "outubro",
    11: "novembro",
    12: "dezembro"
}

def convert_date(date_str):
    # Parse the date string
    date_obj = datetime.strptime(date_str, '%Y-%m-%d')
    
    # Extract the day, month, and year
    day = date_obj.day
    month = month_names[date_obj.month]
    year = date_obj.year
    
    # Format the date in the desired format
    formatted_date = f"{day} de {month} de {year}"
    return formatted_date
    
# Definindo cabeçalhos para a requisição
headers = {
    "Accept": "application/json",
    "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36"
}


def acessar_sinergias(url,headers):
    try:
        # Requisição para obter os dados JSON
        response = requests.get(url, headers=headers)
        response.raise_for_status()  # Verificar se a requisição foi bem-sucedida
    
        # Tentar decodificar o JSON
        data = response.json()
        #print(data)
    
        # Carregar os dados JSON em um DataFrame
        #df = pd.DataFrame(data['despacho'])
        #df['despacho'] = df['despacho'].fillna('Unknown')
    
        #print(df)
    
        # Verificar e converter a coluna 'count' para inteiro
        #df['tempo'] = pd.to_numeric(df['tempo'], errors='coerce')
        return data
    
    except requests.exceptions.HTTPError as http_err:
        print(f"HTTP error occurred: {http_err}")
    except requests.exceptions.RequestException as req_err:
        print(f"Error occurred during request: {req_err}")
    except ValueError as json_err:
        print(f"JSON decode error: {json_err}")
    except Exception as err:
        print(f"An unexpected error occurred: {err}")    
    return -1

numero = '102023024151'
query = '"' + "mysql_query" + '"' ":" + '"' + f" * FROM arquivados where numero='{numero}' and anulado=0 order by data desc" + '"'
url = f"http://www.cientistaspatentes.com.br/apiphp/patents/query/?q={query}"
data = acessar_sinergias(url,headers)

if 'patents' in data and len(data['patents']) > 0 and 'descricao' in data['patents'][0]:
    descricao = data['patents'][0]['descricao']
    print(descricao)

if 'patents' in data and len(data['patents']) > 0 and 'despacho' in data['patents'][0]:
    #despachos = [patent['despacho'] for patent in data['patents']]
    #for despacho in despachos:
    #    print(despacho)
        
    despacho = data['patents'][0]['despacho'].strip()
    formatted_date = convert_date(data['patents'][0]['data'])
    query = '"' + "mysql_query" + '"' ":" + '"' + f" * FROM despachos WHERE despacho='{despacho}'" + '"'
    url = f"http://www.cientistaspatentes.com.br/apiphp/patents/query/?q={query}"
    data = acessar_sinergias(url,headers)
    descricao = data['patents'][0]['descricao'].strip()
    resumo = data['patents'][0]['resumo'].strip()

    print("Última publicação: ",despacho,f"(publicado em {formatted_date})", resumo,'. ', descricao)


Última publicação:  3.1 (publicado em 4 de junho de 2024) Pedido de Patente ou de Certificado de Adição de Invenção Publicado .  Quando o período de sigilo (18 meses do depósito) é finalizado, o INPI publica o pedido de Patente ou de Certificado de Adição de Invenção, conforme previsto no Art. 30 da Lei nº 9.279/96. Os interessados podem acessar a documentação do pedido (relatório descritivo, reivindicações, desenhos - se for o caso - e resumo) na Busca de Processos (Portal do INPI). Não é necessário responder a este despacho. O depositante ou qualquer interessado pode solicitar o exame do pedido de patente, caso isso ainda não tenha sido feito. O prazo é de 36 (trinta e seis) meses, contados da data do depósito. Para tal, o usuário deverá pagar a guia de recolhimento da união (GRU) de código 203 (Patente de Invenção), 204 (Modelo de Utilidade) ou 205 (Certificado de Adição de Invenção). Se o INPI já examinou o pedido internacional PCT como ISA/IPEA, o código é 284 (Patente de Invenção